In [1]:
import asyncio

import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
# client = OpenAI()
client = AsyncOpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = requests.post("https://tasks.aidevs.pl/token/inprompt", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

input_text = response_json['input']
question = response_json['question']
answer = ""
print(question)

jakiego koloru oczy ma Ezaw?


In [5]:
async def asummerize_text(idx, text):
    system_content = """
You are a summary machine that summarizes given text and returns a person name the text is about. You expect the text to be in polish language.

Examples:
user: Adam ma szare oczy, średniej długości włosy i pracuje jako nauczyciel, a na śniadanie najbardziej lubi jeść kurczaka.
asistant: Adam
"""

    completion = await client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": text}
      ]
    )
    return (idx, str(completion.choices[0].message.content))

In [6]:
async def aanswer_question(context, question):
    system_content = """
You are expected to answer given question about some person based on input you are provided. If you cannot figure out the answer simply return 'I do not know that'.

Examples:
user: Context: Kordian ma brązowe oczy, krótkie włosy i pracuje jako architekt. Kordian na śniadanie najbardziej lubi jeść parówki.\n ### \n Question: Jakiego koloru są oczy Kordiana?
asistant: brązowego
"""
    
    completion = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": f"Context: {context}\n #### \n Question: {question}"}
        ]
    )
    return str(completion.choices[0].message.content)

In [7]:
async def main():
    global answer
    q_name = await asummerize_text(0, question)
    print(q_name)
    
    summerize_tasks = [asummerize_text(idx, text) for idx, text in enumerate(input_text)]
    results = await asyncio.gather(*summerize_tasks)
    
    filtered_names = [name_tuple[0] for name_tuple in results if name_tuple[1] == q_name[1]]
    
    context = ""
    for idx, text in enumerate(input_text):
        if idx in filtered_names:
            context = context + input_text[idx] + ". "
    print(context)
    
    answer = await aanswer_question(context, question)
    print(answer)

In [8]:
loop = asyncio.get_event_loop()
# loop.create_task(main())
coroutine = asyncio.run_coroutine_threadsafe(main(), loop)

In [11]:
# Manually check if the async calls to OpenAI are finished...
print(coroutine._state)
print(answer)

FINISHED
niebieskiego


In [12]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=answer))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
